# [Create Agent With OpenAPI](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-openapi)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

Inspired by [sample_agents_openapi.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_openapi.py)

# Constants

In [1]:
import os, jsonref
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
# Create AI Foundry Project Client

In [3]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails # <<<<<<<<<<<<<<< SPECIFIC FOR OPENAPI
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [4]:
# Load Weather OpenAPI

In [5]:
# https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/weather_openapi.json
with open("./openapi/weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

openapi_spec

{'openapi': '3.1.0',
 'info': {'title': 'get weather data',
  'description': 'Retrieves current weather data for a location based on wttr.in.',
  'version': 'v1.0.0'},
 'servers': [{'url': 'https://wttr.in'}],
 'auth': [],
 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location',
    'operationId': 'GetCurrentWeather',
    'parameters': [{'name': 'location',
      'in': 'path',
      'description': 'City or location to retrieve the weather for',
      'required': True,
      'schema': {'type': 'string'}},
     {'name': 'format',
      'in': 'query',
      'description': 'Always use j1 value for this parameter',
      'required': True,
      'schema': {'type': 'string', 'default': 'j1'}}],
    'responses': {'200': {'description': 'Successful response',
      'content': {'text/plain': {'schema': {'type': 'string'}}}},
     '404': {'description': 'Location not found'}},
    'deprecated': False}}},
 'components': {'schemes': {}}}

# Create Auth object for the OpenApiTool 
## Note: connection or managed identity auth setup requires additional setup in Azure

In [6]:
auth = OpenApiAnonymousAuthDetails()
auth

{'type': 'anonymous'}

# Initialize agent OpenApi tool using the read in OpenAPI spec

In [7]:
openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth
)
print(f"openapi.definitions: {openapi.definitions}")
print(f"openapi.resources: {openapi.resources}")

openapi.definitions: [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 'schema': {'type': 'string', 'default': 'j1'}}], 'responses': {'200': {'description': 'Successful response', 'content': {'text/plain': {'schema': {'type': 'string'}}}}, '404': {'description': 'Location not found'}}, 'deprecated': 

# Create AI Foundry Agent

In [8]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="openapi-agent",
    instructions="You are a helpful assistant",
    tools=openapi.definitions
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_Z5DNrbY7TAtVh3pH3lJIm4yU', 'object': 'assistant', 'created_at': 1736732096, 'name': 'openapi-agent', 'description': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': T

# Create the thread and attach a new message to it

In [9]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica esattamente quale è l'ultima data per la quale sei in grado di rispondere.",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'object': 'thread', 'created_at': 1736732097, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_ihJFQTyh1g09eNbLhm0EKsEj', 'object': 'thread.message', 'created_at': 1736732098, 'assistant_id': None, 'thread_id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': "Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica esattamente quale è l'ultima data per la quale sei in grado di rispondere.", 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent synchronously

In [10]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED. Run: {'id': 'run_MDdQgxc96dVCCrTAI9SpCiuU', 'object': 'thread.run', 'created_at': 1736732098, 'assistant_id': 'asst_Z5DNrbY7TAtVh3pH3lJIm4yU', 'thread_id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'status': 'completed', 'started_at': 1736732099, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1736732103, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-0513', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name

# Fetch messages from the thread after the agent run execution

In [11]:
# Fetch messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}\n")

# Fetch the last message from the thread
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message:\n{last_msg.text.value}")

Messages: {'object': 'list', 'data': [{'id': 'msg_uYRzwoY9e78ro0szWlFo8L5H', 'object': 'thread.message', 'created_at': 1736732102, 'assistant_id': 'asst_Z5DNrbY7TAtVh3pH3lJIm4yU', 'thread_id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'run_id': 'run_MDdQgxc96dVCCrTAI9SpCiuU', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "L'ultima data per la quale sono in grado di fornire la temperatura massima a Chiavari è il 15 gennaio 2025. \n\nLa temperatura massima prevista per il 15 gennaio 2025 a Chiavari è di 9°C.", 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_ihJFQTyh1g09eNbLhm0EKsEj', 'object': 'thread.message', 'created_at': 1736732098, 'assistant_id': None, 'thread_id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': "Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica esattamente quale è l'ultima data per la quale sei in grado di ri

In [12]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: L'ultima data per la quale sono in grado di fornire la temperatura massima a Chiavari è il 15 gennaio 2025. 

La temperatura massima prevista per il 15 gennaio 2025 a Chiavari è di 9°C.


# Run Steps

In [13]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_RtVBELQXMRqkK0FpOaXDm7tv', 'object': 'thread.run.step', 'created_at': 1736732102, 'run_id': 'run_MDdQgxc96dVCCrTAI9SpCiuU', 'assistant_id': 'asst_Z5DNrbY7TAtVh3pH3lJIm4yU', 'thread_id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1736732103, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_uYRzwoY9e78ro0szWlFo8L5H'}}, 'usage': {'prompt_tokens': 9218, 'completion_tokens': 56, 'total_tokens': 9274}} 

Run step 2: {'id': 'step_edSB0CO9d5bjQawyllfg4dJF', 'object': 'thread.run.step', 'created_at': 1736732099, 'run_id': 'run_MDdQgxc96dVCCrTAI9SpCiuU', 'assistant_id': 'asst_Z5DNrbY7TAtVh3pH3lJIm4yU', 'thread_id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1736732102, 'expires_at': None, 'failed_at': N

# START teardown

In [14]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'object': 'thread', 'created_at': 1736732097, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_npzLBHwYxFhCeq0fisaOugoS', 'object': 'thread.deleted', 'deleted': True}

In [15]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

1 agent(s) will now be deleted

1 - Agent openapi-agent has been deleted


# HIC SUNT LEONES